In [28]:
import langchain
import json
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from Bio import Entrez

Test LLM

In [29]:
llm = Ollama(model='llama3',temperature=0.1,base_url='http://192.168.0.64:11434')

Generar search queries

In [30]:
symptoms = "Ayer entrené en el gimnasio y tengo un hormigueo que me empieza en la parte baja de la espalda y baja por la pierna derecha."

pm_search_prompt = 'Eres un investigador avanzado en fisioterapia. Tu tarea es buscar papers en PubMed para encontrar un \
                    diagnóstico y tratamiento de fisioterapia para los siguientes síntomas: "{symptoms}" \
                    Dime 10 entradas de búsqueda que pondrías en la web de pubmed para encontrar papers sobre los síntomas. \
                    No emplees combinaciones como OR o AND, dime únicamente las palabras clave de búsqueda.\
                    Devuelve únicamente las entradas de búsqueda como una lista de python, no digas nada más'

pm_search_template = ChatPromptTemplate.from_template(pm_search_prompt)

Function to search pubmed articles

In [31]:
def generate_search(pm_search_template,symptoms,llm):
    pm_search_llm = pm_search_template | llm
    search_entries = pm_search_llm.invoke({'symptoms':symptoms})
    return search_entries
search_entries = generate_search(pm_search_template,symptoms,llm).replace('[','').replace(']','').replace("'","").split(', ')
print(search_entries)

['lower back pain', 'post-exercise soreness', 'delayed onset muscle soreness', 'DOMS', 'lumbar spine', 'gluteal region', 'hip pain', 'leg pain', 'exercise-induced muscle soreness', 'muscle strain', 'spinal stiffness']


Search in pubmed API

In [32]:
def search(entry):
    Entrez.email = 'oscarvallslozano@gmail.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='5',
                            retmode='xml',
                            term=entry)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'oscarvallslozano@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results


results = search(search_entries[0])
id_list = results['IdList']
papers = fetch_details(id_list)
for i, paper in enumerate(papers['PubmedArticle']):
        print("{}) {}".format(i+1, paper['MedlineCitation']['Article']['ArticleTitle']))


1) Neck Pain and Lower Back Pain.
2) Clinical outcomes and cost-effectiveness of massage chair therapy versus basic physiotherapy in lower back pain patients: A randomized controlled trial.
3) Treatment and ergonomics training of work-related lower back pain and body posture problems for nurses.
4) Acute Lumbar Back Pain.
5) Hypnosis As A Therapy for Chronic Lower Back Pain.


Published: --
Title: Research hotspots and trends of complementary and alternative therapy for neuropathic pain: A bibliometric analysis.
Copyright Information: Copyright © 2024 the Author(s). Published by Wolters Kluwer Health, Inc.
Summary::
BACKGROUND: Neuropathic pain (NP) is a common type of pain in clinic. Due to the limited effect of drug treatment, many patients with NP are still troubled by this disease. In recent years, complementary and alternative therapy (CAT) has shown good efficacy in the treatment of NP. As the interest in CAT for NP continues to grow, we conducted a bibliometric study of publications on CAT treatment for NP. The aim of this study is to analyze the development overview, research hotspots and future trends in the field of CAT and NP through bibliometric methodology, so as to provide a reference for subsequent researchers.
METHODS: Publications on CAT in the treatment of NP from 2002 to 2022 were retrieved from the Web of Science Core Collection. Relevant